# Cycling in numbers - A Case Study of Cycle Paths in Rhine-Kreis Neuss

## Description

Five Counting stations have been permanently documenting cycling traffic on central roads since 2016 in Cycle paths in the Rhine-Kreis Neuss. The daily measurement of cycling traffic is done with the help of induction loops laid in the ground. With the permanent collection of data can gain insights on the daily, weekly and annual cycles and on it building long-term cycling developments over several years.

More details on the data source here: https://data.europa.eu/data/datasets/eco-counter-data-rhein-kreis-neuss?locale=en

# Streaming Simulation - Data Ingestion

To simulate real-time data ingestion, Apache Kafka is used to stream bicycle count data continuously. A Kafka producer reads records from the dataset and publishes them as JSON messages to a designated topic, replicating live data collection scenarios. The simulation introduces controlled time delays, mimicking the hourly arrival of new data.


Each message follows a predefined schema to maintain data integrity. Logging and monitoring mechanisms track message flow, ensuring successful ingestion and identifying potential issues. This simulation validates the system's ability to handle real-time data, laying the groundwork for further streaming analytics and dynamic monitoring applications.



In [1]:
from kafka import KafkaProducer
import pandas as pd
import time
import json
from datetime import datetime
from pyspark.sql.session import SparkSession
import findspark


findspark.init()

import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2" pyspark-shell'

pd.set_option('display.max_colwidth', None)

spark = (SparkSession.builder
    .appName("group_10_streaming_simulation")
    .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

# Kafka broker
KAFKA_BROKER = 'localhost:9092'
KAFKA_TOPIC = 'cycling_data'

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=KAFKA_BROKER,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# HDFS CSV path
df = (spark.read.option("header", "true")
                 .option("inferSchema", "true")
                 .csv("hdfs://localhost:9000//g10_datalake/silver/cycling/Counting_Station_Meerbusch")
                 .cache())


df = df.orderBy("Date")

# Simulate hourly streaming by sending rows one by one
for row in df.collect():
    data = row.asDict() 
    
    # Convert datetime objects to string
    
    for key, value in data.items():
        if isinstance(value, datetime):
            data[key] = value.isoformat() 
    producer.send(KAFKA_TOPIC, value=data)
    print(f"Sent: {data}")
    
    # Adjust for hourly simulation (2 second = 1 "hour")
    time.sleep(2)  

# Close producer
producer.close()

25/03/17 18:33:01 WARN Utils: Your hostname, osbdet resolves to a loopback address: 127.0.0.1; using 10.0.2.15 instead (on interface enp0s3)
25/03/17 18:33:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/osbdet/.jupyter_venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/osbdet/.ivy2/cache
The jars for the packages stored in: /home/osbdet/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-89591676-f88b-4b1d-96a1-cd4bf298902b;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.2 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 2613ms :: artifacts dl 113ms
	

Sent: {'Id': 100019715, 'Date': '2016-01-26T00:00:00', 'Count': 0, 'Status': 'raw', 'Channel_Id': 101019715, 'Counting_Station': 'Meerbusch', 'Coordinates': '51.261012, 6.705018', 'Year': 2016, 'Month': 1, 'Day': 26, 'Hour': 0, 'Day_of_Week': 3, 'Latitude': 51.261012, 'Longitude': 6.705018}


Sent: {'Id': 100019715, 'Date': '2016-01-26T00:00:00', 'Count': 0, 'Status': 'raw', 'Channel_Id': 102019715, 'Counting_Station': 'Meerbusch', 'Coordinates': '51.261012, 6.705018', 'Year': 2016, 'Month': 1, 'Day': 26, 'Hour': 0, 'Day_of_Week': 3, 'Latitude': 51.261012, 'Longitude': 6.705018}


...

Sent: {'Id': 100019715, 'Date': '2016-01-26T21:00:00', 'Count': 0, 'Status': 'raw', 'Channel_Id': 102019715, 'Counting_Station': 'Meerbusch', 'Coordinates': '51.261012, 6.705018', 'Year': 2016, 'Month': 1, 'Day': 26, 'Hour': 21, 'Day_of_Week': 3, 'Latitude': 51.261012, 'Longitude': 6.705018}


Sent: {'Id': 100019715, 'Date': '2016-01-26T22:00:00', 'Count': 0, 'Status': 'raw', 'Channel_Id': 101019715, 'Counting_Station': 'Meerbusch', 'Coordinates': '51.261012, 6.705018', 'Year': 2016, 'Month': 1, 'Day': 26, 'Hour': 22, 'Day_of_Week': 3, 'Latitude': 51.261012, 'Longitude': 6.705018}


Sent: {'Id': 100019715, 'Date': '2016-01-26T22:00:00', 'Count': 0, 'Status': 'raw', 'Channel_Id': 103019715, 'Counting_Station': 'Meerbusch', 'Coordinates': '51.261012, 6.705018', 'Year': 2016, 'Month': 1, 'Day': 26, 'Hour': 22, 'Day_of_Week': 3, 'Latitude': 51.261012, 'Longitude': 6.705018}


Sent: {'Id': 100019715, 'Date': '2016-01-26T22:00:00', 'Count': 0, 'Status': 'raw', 'Channel_Id': 104019715, 'Counting_Station': 'Meerbusch', 'Coordinates': '51.261012, 6.705018', 'Year': 2016, 'Month': 1, 'Day': 26, 'Hour': 22, 'Day_of_Week': 3, 'Latitude': 51.261012, 'Longitude': 6.705018}


KeyboardInterrupt: 